In [ ]:
import pandas as pd
import numpy as np
from data_fetch import fetch 
from volume_indicator import rvol_daily

SYMBOL = "CRCL"
TZ = "America/New_York"
RVOL_LOOKBACK_DAYS = 20
ATR_LOOKBACK_DAYS = 14

raw_df = fetch("CRCL", "1d", period="1y", tz="America/New_York", prepost=False)

In [35]:
def atr(df: pd.DataFrame, window: int = 14, mode: str = "backtest") -> pd.Series:
    """
    Compute ATR (Average True Range).

    Parameters
    ----------
    df : DataFrame with ['High','Low','Close'].
    window : int, lookback period.
    mode : 'backtest' → shift(1) to avoid lookahead,
           'live' → include current bar.

    Returns
    -------
    pd.Series of ATR values.
    """
    high, low, close = df["High"], df["Low"], df["Close"]

    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    atr = tr.rolling(window, min_periods=1).mean()

    if mode == "backtest":
        atr = atr.shift(1)

    return atr.rename(f"ATR_{window}")

In [36]:
def rvol(df: pd.DataFrame, window: int = 20, mode: str = "backtest") -> pd.Series:
    """
    Compute Relative Volume (RVOL).

    Parameters
    ----------
    df : DataFrame with ['Volume'].
    window : int, lookback period.
    mode : 'backtest' → shift(1) to avoid lookahead,
           'live' → include current bar.

    Returns
    -------
    pd.Series of RVOL values (float).
    """
    avg_vol = df["Volume"].rolling(window, min_periods=max(1, window // 2)).mean()

    if mode == "backtest":
        avg_vol = avg_vol.shift(1)

    rvol = df["Volume"] / avg_vol.replace(0, np.nan)
    return rvol.rename(f"RVOL_{window}")

In [37]:
def price_deviation(df: pd.DataFrame, window: int = 20, mode: str = "backtest") -> pd.Series:
    """
    Calculate z-score style price deviation.

    Parameters
    ----------
    df : DataFrame with 'Close'.
    window : int, lookback period.
    mode : 'backtest' → exclude current bar,
           'live' → include current bar.

    Returns
    -------
    pd.Series with deviation values.
    """
    mean = df["Close"].rolling(window).mean()
    std = df["Close"].rolling(window).std()

    if mode == "backtest":
        mean = mean.shift(1)
        std = std.shift(1)

    return ((df["Close"] - mean) / std).rename(f"PriceDev_{window}")

In [39]:
atr_series = atr(raw_df, ATR_LOOKBACK_DAYS, mode="backtest")
rvol_series = rvol(raw_df, RVOL_LOOKBACK_DAYS, mode="backtest")
sigma_series = price_deviation(raw_df, ATR_LOOKBACK_DAYS, mode="backtest")
### attach them into one dataframe
combined_df = pd.concat([atr_series, rvol_series, sigma_series], axis=1)
combined_df.columns = ["ATR", "RVOL", "Price_Deviation"]
combined_df

,ATR,RVOL,Price_Deviation
Date,,,
2025-06-04 20:00:00-04:00,NaN,NaN,NaN
2025-06-05 20:00:00-04:00,39.750000,NaN,NaN
2025-06-08 20:00:00-04:00,40.017498,NaN,NaN
2025-06-09 20:00:00-04:00,36.968335,NaN,NaN
2025-06-10 20:00:00-04:00,31.161251,NaN,NaN
2025-06-11 20:00:00-04:00,27.845000,NaN,NaN
2025-06-12 20:00:00-04:00,25.482666,NaN,NaN
2025-06-15 20:00:00-04:00,25.867999,NaN,NaN
2025-06-16 20:00:00-04:00,26.639501,NaN,NaN
